# Central Schemes 

This notebook includes Python code to solve conservation laws of the form $\partial_t u + \nabla f(u) = 0$
using central schemes. The schemes are translated into Python from [CentPack](https://home.cscamm.umd.edu/centpack/) written by [Jorge Balbás](http://www.csun.edu/~jb715473/index.htm) and [Eitan Tadmor](https://home.cscamm.umd.edu/people/faculty/tadmor/).

In [ ]:
# Configuration and packages

### This block is just to make the inline figures look good
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.dpi']  = 60

### Packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

## CentPy 1D

### Schemes

In [ ]:
# Common functions: minmod, minmod3, minmod_prime, bc_periodic_1d, H_half, c_flux

# Index definitions for convenience
j0=slice(2,-2)
jp=slice(3,-1)
jm=slice(1,-3)

def minmod(a,b):
    return 0.5*(np.sign(a)+np.sign(b))*np.minimum(np.abs(a),np.abs(b))

def minmod3(a,b,c):
    return minmod(a,minmod(b,c))

def minmod_prime(u):
    return minmod3(alpha*(u[1:-1] - u[:-2]), 0.5*(u[2:] - u[:-2]), alpha*(u[2:]- u[1:-1]))

def va(a,b):
    c = a**2+b**2
    return np.divide( a*b*(a+b), c , out=np.zeros_like(a), where=c!=0)

def va_prime(u):
    return va(u[2:]-u[1:-1],u[1:-1]-u[:-2])

def va_eps(a,b):
    eps = 1e-10
    return ((a**2+eps**2)*b+(b**2+eps**2)*a)/(a**2+b**2+2*eps**2)

def va_prime_eps(u):
    return va_eps(u[2:]-u[1:-1],u[1:-1]-u[:-2])

def bc_periodic_1d(u):
    u[0] = u[-4]
    u[1] = u[-3]
    u[-2] = u[2]
    u[-1] = u[3]
    
def H_half(u_E,u_W):
    a = np.maximum(spectral_radius(u_E), spectral_radius(u_W))
    f_E = flux(u_E)
    f_W = flux(u_W)
    if u_W.shape==a.shape:
        return 0.5*(f_W+f_E) - 0.5*a*(u_W-u_E) # scalar
    else:
        return 0.5*(f_W+f_E) - 0.5*np.multiply(a[:,None],(u_W-u_E)) # systems

def c_flux(u_E,u_W):
    H_halfm = H_half(u_E[jm],u_W[j0])
    H_halfp = H_half(u_E[j0],u_W[jp])
    return -dt/dx*(H_halfp-H_halfm)

In [ ]:
# FD2 

def fd2(u):
    global odd
    u_prime = np.ones(u.shape)
    uj_half = np.ones(u.shape)
    un_half = np.ones(u.shape)
    boundary_conditions(u)
    f=flux(u)
    u_prime[1:-1] = limiter(u)
    # Predictor
    un_half[1:-1] = u[1:-1] - 0.5*dt/dx*limiter(f)
    f_half = flux(un_half)
    # Corrector
    if(odd):
        u[1:-2] = 0.5*(u[2:-1] + u[1:-2]) + .125*(u_prime[1:-2] - u_prime[2:-1]) \
            - dt/dx*(f_half[2:-1] - f_half[1:-2])
    else:
        u[2:-1] = 0.5*(u[2:-1] + u[1:-2]) + .125*(u_prime[1:-2] - u_prime[2:-1]) \
            - dt/dx*(f_half[2:-1] - f_half[1:-2])
    # Boundary conditions
    boundary_conditions(u)
    # Switch
    odd=not odd
    return u

In [ ]:
# SD2

def reconstruction_sd2(u):
    # Reconstruction
    u_E = np.ones(u.shape)
    u_W = np.ones(u.shape)
    s = limiter(u[1:-1])
    u_E[j0] = u[j0] + 0.5*s
    u_W[j0] = u[j0] - 0.5*s
    boundary_conditions(u_E)
    boundary_conditions(u_W)
    return u_E, u_W
    
def sd2(u):
    boundary_conditions(u)
    u_E, u_W = reconstruction_sd2(u)
    C0=c_flux(u_E, u_W)
    u[j0] += C0
    boundary_conditions(u)
    u_E, u_W = reconstruction_sd2(u)
    C1=c_flux(u_E, u_W)
    u[j0] += 0.5*(C1-C0)
    boundary_conditions(u)
    return u

In [ ]:
# SD3

def p_coefs(u):
    pl0 = u[j0]
    pl1 = u[j0]-u[jm]
    pr0 = u[j0]
    pr1 = u[jp]-u[j0]
    pc0 = pl0 - (1./12.)*(pr1-pl1)
    pc1 = 0.5*(pl1+pr1)
    pc2 = pr1-pl1;
    return pl0, pl1, pr0, pr1, pc0, pc1, pc2
    
def reconstruction_sd3(u, ISl, ISc, ISr):
    cl=.25; cc=.5; cr=.25;    
    alpl=cl/((eps+ISl)*(eps+ISl))
    alpc=cc/((eps+ISc)*(eps+ISc))
    alpr=cr/((eps+ISr)*(eps+ISr))
    alp_sum=alpl+alpc+alpr
    wl=alpl/alp_sum;
    wc=alpc/alp_sum;
    wr=alpr/alp_sum;
    pl0, pl1, pr0, pr1, pc0, pc1, pc2 = p_coefs(u)
    u_E = np.ones(u.shape)
    u_W = np.ones(u.shape)
    u_E[j0]=wl*(pl0+.5*pl1) + wc*(pc0+.5*pc1+.25*pc2) +wr*(pr0+.5*pr1)
    u_W[j0]=wl*(pl0-.5*pl1) + wc*(pc0-.5*pc1+.25*pc2) +wr*(pr0-.5*pr1)    
    # boundary
    boundary_conditions(u_E)
    boundary_conditions(u_W)
    return u_E, u_W
    
def sd3(u): # Implemented for one equation (for systems modify IS declarations)
    boundary_conditions(u)
    u_norm = np.sqrt(dx)*np.linalg.norm(u[j0])
    pl0, pl1, pr0, pr1, pc0, pc1, pc2 = p_coefs(u)
    ISl = pl1*pl1/(u_norm+eps)
    ISc = 1./(u_norm+eps)*((13./3.)*pc2*pc2 + pc1*pc1)
    ISr = pr1*pr1/(u_norm+eps)
    u_E, u_W = reconstruction_sd3(u, ISl, ISc, ISr)
    C0=c_flux(u_E, u_W)
    u[2:-2] += + C0
    boundary_conditions(u)
    u_E, u_W = reconstruction_sd3(u, ISl, ISc, ISr)
    C1=c_flux(u_E, u_W)
    u[j0] += + 0.25*(C1-3.*C0)    
    boundary_conditions(u)
    u_E, u_W = reconstruction_sd3(u, ISl, ISc, ISr)
    C2=c_flux(u_E, u_W)
    u[j0] += + 1./12.*(8.*C2-C1-C0)
    boundary_conditions(u)
    return u

### Examples

#### Burgers (SD3)

We solve the inviscid Burgers' equation 

\begin{equation}
\partial_t u + \partial_x \left( \frac{u^2}{2}\right) = 0,
\end{equation}

on the domain $(x,t)\in([0,2\pi]\times[0,5])$ with initial data

\begin{equation}
u(x,0) = \sin x + \frac{1}{2} \sin\left(\frac{x}{2}\right)\,. 
\end{equation}

and periodic boundary conditions. The solution is computed using 400 cells and CFL number 0.75.

In [ ]:
# Parameters

x_init, x_final = 0.0, 2*np.pi # start and end of spatial domain
J = 400
L=1
gamma=1.4
t_final = 5.0
dt_out = 0.05
cfl = 0.75
alpha = 1.4

# Space grid
dx=(x_final - x_init)/J
x = np.linspace(x_init-2*dx, x_final+dx, J+4)

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

# Scheme-specific parameters
odd=True # for FD2
eps=0.000001 # for SD3

In [ ]:
# Functions

def initial_burgers_1d():
    return np.sin(x) + 0.5*np.sin(0.5*x)        

bc_burgers_1d = bc_periodic_1d

def flux_burgers_1d(u):
    return 0.5*u*u 

def spectral_radius_burgers_1d(u):
    return np.abs(u)

initial=initial_burgers_1d
boundary_conditions=bc_periodic_1d
flux=flux_burgers_1d
spectral_radius=spectral_radius_burgers_1d

In [ ]:
# Simulation

solver=sd3

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max = np.max(spectral_radius(u))
    dt=dx*cfl/r_max
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation 
 
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(np.min(u_n), np.max(u_n)*1.1))

line_u, = ax.plot([], [], linewidth=1, marker='o', markersize=2)

plt.xlabel('x')
plt.ylabel('u')
plt.legend(["burgers"], loc=3, frameon=False)
 
# initialization function: plot the background of each frame
def init():
    line_u.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    line_u.set_data(x[j0], u_n[i,j0])
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=Nt, interval=100, blit=False)
 
plt.show()

#### Euler (SD2)

We solve the Euler equations in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} \rho \\ \rho v \\ E \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} \rho v \\ \rho v^2 +p \\ v (E+p) \end{bmatrix} 
= 0 
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho v^2\right), \qquad \gamma=1.4
\end{equation}

on the domain $(x,t)\in([0,1]\times[0,0.1])$ with initial data for a *Sod shock tube*:

\begin{equation}
(\rho, v, p)_{t=0} = 
\begin{cases}
(1,0,1) & \text{if} & 0<x\leq0.5 \\
(0.125, 0, 0.1) & \text{if} & 0.5<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using 400 cells and CFL number 0.75.

In [ ]:
# Parameters
x_init, x_final = 0.0, 1.0 # start and end of spatial domain
J = 400 # number of spatial points
L=3
gamma=1.4
t_final = 0.3
dt_out = 0.004
cfl = 0.75
alpha = 0.5*(1.+np.sqrt(2))
limiter=minmod_prime

# Space grid
dx=(x_final - x_init)/J
#x = np.linspace(x_init-2*dx, x_final+dx, J+4)
x = np.linspace(x_init-1.5*dx, x_final+1.5*dx, J+4) # staggered grid

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

# Scheme-specific parameters
odd=True # for FD2
eps=0.000001 # for SD3

In [ ]:
# Functions

def initial_euler_1d():
    u=np.empty((J+4,L))
    midpoint = int(J/2)+2

    left_v =  [1,0,1./(gamma-1.)]
    right_v = [0.125, 0., 0.1/(gamma-1.)]

    # Left side
    u[:midpoint,:] = left_v
    # Right side
    u[midpoint:,:] = right_v

    return u

def bc_euler_1d(u):
    
    left_v =  [1,0,1./(gamma-1.)]
    right_v = [0.125, 0., 0.1/(gamma-1.)]
    # Left side
    u[0] = left_v
    u[1] = left_v
    # Right side
    u[-1] = right_v
    u[-2] = right_v    
    return u

def flux_euler_1d(u):
    f = np.empty_like(u)

    rho=u[:,0]
    u_x = u[:,1]/rho
    E = u[:,2]
    p = (gamma-1.)*(E-0.5*rho*u_x**2)

    f[:,0] = rho*u_x
    f[:,1] = rho*u_x**2+p
    f[:,2] = u_x*(E+p)

    return f

def spectral_radius_euler_1d(u):
    rho=u[:,0]  
    u_x = u[:,1]/rho
    p = (gamma-1.)*(u[:,2]-0.5*rho*u_x**2)
    return np.abs(u_x) + np.sqrt(gamma*p/rho)

initial=initial_euler_1d
boundary_conditions=bc_euler_1d
flux=flux_euler_1d
spectral_radius=spectral_radius_euler_1d

In [ ]:
# Euler in 1D

solver=sd2

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max = np.max(spectral_radius(u))
    dt=dx*cfl/r_max
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation 
 
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(10,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(2,2,2)
ax3=fig.add_subplot(2,2,4)
#ax = plt.axes(xlim=(x_init,x_final), ylim=(np.min(u_n), np.max(u_n)*1.1))

line_u1, = ax1.plot([], [], linewidth=1, marker='o', markersize=2)
line_u2, = ax2.plot([], [], linewidth=1, marker='o', markersize=2)
line_u3, = ax3.plot([], [], linewidth=1, marker='o', markersize=2)

ax1.set_xlabel('x')
ax3.set_xlabel('x')
ax1.set_ylabel(r'$\rho$')
ax2.set_ylabel(r'$v_x$')
ax3.set_ylabel(r'$p$')
ax1.set_ylim(0.1, 1.05)
ax2.set_ylim(0, 1)
ax3.set_ylim(0.05, 1.1)
#plt.legend(["u0","u1","u2"], loc=3, frameon=False)

plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)

# initialization function: plot the background of each frame
def init():
    line_u1.set_data([], [])
    line_u2.set_data([], [])
    line_u3.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    rho=u_n[i,j0,0]
    u_x = u_n[i,j0,1]/u_n[i,j0,0]
    p = (gamma-1.)*(u_n[i,j0,2]-0.5*rho*u_x**2)
    line_u1.set_data(x[j0], rho)
    line_u2.set_data(x[j0], u_x)
    line_u3.set_data(x[j0], p)
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=Nt, interval=100, blit=False)
 
plt.show()

#### MHD (FD2)

We solve the equations of ideal magnetohydrodynamics in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} 
\rho \\ \rho v_x \\ \rho v_y \\ \rho v_z \\ B_y \\ B_z \\ E 
\end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} 
\rho v_x \\ \rho v_x^2 + p^* - B_x^2 \\ \rho v_x v_y - B_x B_y \\
\rho v_x v_z - B_x B_z \\ B_y v_x - B_x v_y \\ B_z v_x - B_x v_z \\
(E+p^*) v_x - B_x (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
= 0 
\end{equation}

where the total pressure is given by 

\begin{equation}
p^* = p + \frac{1}{2} (B_x^2 + B_y^2 + B_z^2)
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (v_x^2+v_y^2+v_z^2) - 
\frac{1}{2}(B_x^2 + B_y^2 + B_z^2)\right), \qquad \gamma=2.0
\end{equation}

The solution is computed on the domain $(x,t)\in([-1,1]\times[0,0.2])$ with initial data for a *Brio-Wu shock tube*:

\begin{equation}
(\rho, v_x, v_y, v_z, B_y, B_z, p)_{t=0} = 
\begin{cases}
(1,0,0,0,1,0,1) & \text{if} & -1<x\leq 0 \\
(0.125, 0, 0, 0, -1, 0, 0.1) & \text{if} & \ \ 0<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using 400 cells and CFL number 0.475.

In [ ]:
# Parameters
x_init, x_final = -1.0, 1.0 # start and end of spatial domain
J = 400 # number of spatial points
L=7
gamma=2.0
t_final = 0.2
dt_out = 0.002
cfl = 0.475
alpha = 1.4
B1=0.75

# Space grid
dx=(x_final - x_init)/J
x = np.linspace(x_init-2*dx, x_final+dx, J+4)

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

# Scheme-specific parameters
odd=True # for FD2
eps=0.000001 # for SD3

In [ ]:
# Functions

def pressure(u):
    return u[:,6]-0.5*((u[:,1]**2+u[:,2]**2+u[:,3]**2)/u[:,0])-0.5*(B1**2+u[:,4]**2+u[:,5]**2)

def initial_mhd_1d():
    u=np.empty((J+4,L))
    midpoint = int(J/2)+2
    
    # Left side
    u[:midpoint,0] = 1.
    u[:midpoint,1] = 0.
    u[:midpoint,2] = 0.
    u[:midpoint,3] = 0.
    u[:midpoint,4] = 1.
    u[:midpoint,5] = 0.
    u[:midpoint,6] = 1.+25./32.
    
    # Right side
    u[midpoint:,0] = 0.125
    u[midpoint:,1] = 0.
    u[midpoint:,2] = 0.
    u[midpoint:,3] = 0.
    u[midpoint:,4] = -1.
    u[midpoint:,5] = 0.
    u[midpoint:,6] = 0.1+25./32.

    return u    

def bc_mhd_staggered_1d(u):
    
    left_v = [1., 0.,0.,0.,1.,0.,1.+25./32.]
    right_v = [0.125,0.,0.,0.,-1.,0.,.1+25./32]

    if(odd):
        u[0] = left_v
        u[-1]= right_v
        u[-2]= right_v
    else:
        u[0] = left_v
        u[1] = left_v
        u[-1]= right_v
        
    return u    

def bc_mhd_1d(u):
    
    left_v = [1., 0.,0.,0.,1.,0.,1.+25./32.]
    right_v = [0.125,0.,0.,0.,-1.,0.,.1+25./32]

    u[0] = left_v
    u[1] = left_v
    u[-1]= right_v
    u[-2]= right_v

    return u    

def flux_mhd_1d(u):
    f = np.empty_like(u)

    p_star = pressure(u)+0.5*(B1**2+u[:,4]**2+u[:,5]**2)

    f[:,0] = u[:,1]
    f[:,1] = u[:,1]**2/u[:,0]+p_star
    f[:,2] = u[:,1]*u[:,2]/u[:,0]-B1*u[:,4]
    f[:,3] = u[:,1]*u[:,3]/u[:,0]-B1*u[:,5]
    f[:,4] = u[:,1]*u[:,4]/u[:,0]-B1*u[:,2]/u[:,0]
    f[:,5] = u[:,1]*u[:,5]/u[:,0]-B1*u[:,3]/u[:,0]
    f[:,6] = (u[:,6]+p_star)*(u[:,1]/u[:,0]) -B1*(B1*u[:,1]+u[:,2]*u[:,4]+u[:,3]*u[:,5])/u[:,0]
    
    return f

def spectral_radius_mhd_1d(u):
    rho=u[:,0]  
    u_x = u[:,1]/rho
    p = pressure(u)
    A = 2.0*p/rho
    B = (B1**2 + u[:,4]**2 + u[:,5]**2)/rho
    cf = np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4.*A*B1**2/rho)))
    return np.abs(u_x) + cf

initial=initial_mhd_1d
flux=flux_mhd_1d
spectral_radius=spectral_radius_mhd_1d
boundary_conditions=bc_mhd_staggered_1d

In [ ]:
# MHD in 1D

solver=fd2

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max = np.max(spectral_radius(u))
    dt=dx*cfl/r_max
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation 
 
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(12,6))
ax1=fig.add_subplot(1,3,1)
ax2=fig.add_subplot(2,3,2)
ax3=fig.add_subplot(2,3,3)
ax4=fig.add_subplot(2,3,5)
ax5=fig.add_subplot(2,3,6)
#ax = plt.axes(xlim=(x_init,x_final), ylim=(np.min(u_n), np.max(u_n)*1.1))

line_u1, = ax1.plot([], [], linewidth=1, marker='o', markersize=2)
line_u2, = ax2.plot([], [], linewidth=1, marker='o', markersize=2)
line_u3, = ax3.plot([], [], linewidth=1, marker='o', markersize=2)
line_u4, = ax4.plot([], [], linewidth=1, marker='o', markersize=2)
line_u5, = ax5.plot([], [], linewidth=1, marker='o', markersize=2)

ax1.set_xlabel('x')
ax4.set_xlabel('x')
ax5.set_xlabel('x')

ax2.set_xticks([])
ax3.set_xticks([])
ax2.set_yticks([])
ax3.set_yticks([])
ax4.set_yticks([])
ax5.set_yticks([])

ax1.set_ylabel(r'$\rho$', fontsize=12)
ax2.set_ylabel(r'$v_x$', fontsize=12)
ax3.set_ylabel(r'$v_y$', fontsize=12)
ax4.set_ylabel(r'$B_y$', fontsize=12)
ax5.set_ylabel(r'$p$', fontsize=12)

# Primitive variables
rho=u_n[:,j0,0]
v_x = u_n[:,j0,1]/u_n[:,j0,0]
v_y = u_n[:,j0,2]/u_n[:,j0,0]
B_y = u_n[:,j0,4]

ax1.set_xlim(-1, 1)
ax2.set_xlim(-1, 1)
ax3.set_xlim(-1, 1)
ax4.set_xlim(-1, 1)
ax5.set_xlim(-1, 1)
ax1.set_ylim(np.min(rho), np.max(rho))
ax2.set_ylim(np.min(v_x), np.max(v_x))
ax3.set_ylim(np.min(v_y), np.max(v_y))
ax4.set_ylim(np.min(B_y), np.max(B_y))
ax5.set_ylim(0.05, 1.)

#plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)

# initialization function: plot the background of each frame
def init():
    line_u1.set_data([], [])
    line_u2.set_data([], [])
    line_u3.set_data([], [])
    line_u4.set_data([], [])
    line_u5.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    p = pressure(u_n[i,j0,:])
    line_u1.set_data(x[j0], rho[i])
    line_u2.set_data(x[j0], v_x[i])
    line_u3.set_data(x[j0], v_y[i])
    line_u4.set_data(x[j0], B_y[i])
    line_u5.set_data(x[j0], p)
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=Nt, interval=100, blit=False)
 
plt.show()

## CentPy 2D

### Schemes

In [ ]:
# Common functions minmod_prime_x, minmod_prime_y

def minmod(a,b):
    return 0.5*(np.sign(a)+np.sign(b))*np.minimum(np.abs(a),np.abs(b))

def minmod3(a,b,c):
    return minmod(a,minmod(b,c))

def minmod_prime(u):
    return minmod3(alpha*(u[1:-1] - u[:-2]), 0.5*(u[2:] - u[:-2]), alpha*(u[2:]- u[1:-1]))

def minmod_prime_x(u):
    return minmod3(alpha*(u[1:-1,1:-1] - u[:-2,1:-1]), 0.5*(u[2:,1:-1] - u[:-2,1:-1]), alpha*(u[2:,1:-1]- u[1:-1,1:-1]))

def minmod_prime_y(u):
    return minmod3(alpha*(u[1:-1,1:-1] - u[1:-1,:-2]), 0.5*(u[1:-1,2:] - u[1:-1,:-2]), alpha*(u[1:-1,2:]- u[1:-1,1:-1]))

In [ ]:
# FD2

def fd2_2d(u):
    global odd
    un_half=np.ones(u.shape)
    u_star =np.ones(u.shape)
    un_half=np.ones(u.shape)
    u_prime_x=np.ones(u.shape)
    u_prime_y=np.ones(u.shape)

#    boundary_conditions(u)
    u_prime_x[1:-1,1:-1]=minmod_prime_x(u)
    u_prime_y[1:-1,1:-1]=minmod_prime_y(u)

    if(odd):
        un_half[1:-2,1:-2]=0.25*((u[1:-2,1:-2] + u[2:-1,1:-2] + u[1:-2,2:-1] + u[2:-1,2:-1]) + \
                                0.25*((u_prime_x[1:-2,1:-2] - u_prime_x[2:-1,1:-2]) + \
                                      (u_prime_x[1:-2,2:-1] - u_prime_x[2:-1,2:-1]) + \
                                      (u_prime_y[1:-2,1:-2] - u_prime_y[1:-2,2:-1]) + \
                                      (u_prime_y[2:-1,1:-2] - u_prime_y[2:-1,2:-1]) ) )
    else:
        un_half[2:-1,2:-1]=0.25*((u[1:-2,1:-2] + u[2:-1,1:-2] + u[1:-2,2:-1] + u[2:-1,2:-1]) + \
                                0.25*((u_prime_x[1:-2,1:-2] - u_prime_x[2:-1,1:-2]) + \
                                      (u_prime_x[1:-2,2:-1] - u_prime_x[2:-1,2:-1]) + \
                                      (u_prime_y[1:-2,1:-2] - u_prime_y[1:-2,2:-1]) + \
                                      (u_prime_y[2:-1,1:-2] - u_prime_y[2:-1,2:-1]) ) )

    f = flux_x(u)
    g = flux_y(u)

    f_prime_x = minmod_prime_x(f)
    g_prime_y = minmod_prime_y(g)
    
    u_star[1:-1,1:-1] = u[1:-1,1:-1] - 0.5*dt*(f_prime_x/dx + g_prime_y/dy)
    boundary_conditions(u_star)
    f_star = flux_x(u_star)
    g_star = flux_y(u_star)

    if(odd):
        u[1:-2,1:-2]= un_half[1:-2,1:-2] - \
                        0.5*dt/dx*((f_star[2:-1,1:-2]-f_star[1:-2,1:-2])+(f_star[2:-1,2:-1]-f_star[1:-2,2:-1])) - \
                        0.5*dt/dy*((g_star[1:-2,2:-1]-g_star[1:-2,1:-2])+(g_star[2:-1,2:-1]-g_star[2:-1,1:-2]))
    else:
        u[2:-1,2:-1]= un_half[2:-1,2:-1] - \
                        0.5*dt/dx*((f_star[2:-1,1:-2]-f_star[1:-2,1:-2])+(f_star[2:-1,2:-1]-f_star[1:-2,2:-1])) - \
                        0.5*dt/dy*((g_star[1:-2,2:-1]-g_star[1:-2,1:-2])+(g_star[2:-1,2:-1]-g_star[2:-1,1:-2]))

    boundary_conditions(u)
    odd=not odd
    return u

In [ ]:
# SD2

j=slice(2,-2)
jp=slice(3,-1)
jm=slice(1,-3)

def reconstruction_sd2_2d(u):
    u_N,u_S,u_E,u_W = np.ones((4,)+u.shape)
    ux=minmod_prime_x(u[1:-1,1:-1])
    uy=minmod_prime_y(u[1:-1,1:-1])
    u_N[j,j],u_S[j,j],u_E[j,j],u_W[j,j]=(u[None,j,j]+np.array([0.5*uy,-0.5*uy,0.5*ux,-0.5*ux]))
    list(map(boundary_conditions,[u_N,u_S,u_E,u_W]))
    return u_N, u_S, u_E, u_W
    
# def H_flux(u_E, u_W, flux, spectral_radius):
#     a = np.maximum(spectral_radius(u_E),spectral_radius(u_W))
#     f_E = flux(u_E)
#     f_W = flux(u_W)
#     if u_W.shape==a.shape:
#         return 0.5*(f_W+f_E) - 0.5*a*(u_W-u_E) # scalar
#     else:
#         return 0.5*(f_W+f_E) - 0.5*np.multiply(a[:,:,None],(u_W-u_E)) # systems

def Hx_flux_sd2_2d(u_E, u_W):
    a = np.maximum(spectral_radius_x(u_E),spectral_radius_x(u_W))
    f_E = flux_x(u_E)
    f_W = flux_x(u_W)
    if u_W.shape==a.shape:
        return 0.5*(f_W+f_E) - 0.5*a*(u_W-u_E) # scalar
    else:
        return 0.5*(f_W+f_E) - 0.5*np.multiply(a[:,:,None],(u_W-u_E)) # systems

def Hy_flux_sd2_2d(u_E, u_W):
    a = np.maximum(spectral_radius_y(u_E),spectral_radius_y(u_W))
    f_E = flux_y(u_E)
    f_W = flux_y(u_W)
    if u_W.shape==a.shape:
        return 0.5*(f_W+f_E) - 0.5*a*(u_W-u_E) # scalar
    else:
        return 0.5*(f_W+f_E) - 0.5*np.multiply(a[:,:,None],(u_W-u_E)) # systems

def c_flux_sd2_2d(u_N,u_S,u_E,u_W):
    Hx_halfm = Hx_flux_sd2_2d(u_E[jm,j],u_W[j,j])
    Hx_halfp = Hx_flux_sd2_2d(u_E[j,j],u_W[jp,j])
    Hy_halfm = Hy_flux_sd2_2d(u_N[j,jm],u_S[j,j])
    Hy_halfp = Hy_flux_sd2_2d(u_N[j,j],u_S[j,jp])
    return - dt/dx * (Hx_halfp - Hx_halfm) - dt/dy * (Hy_halfp - Hy_halfm)

def sd2_2d(u):
    boundary_conditions(u)
    u_N,u_S,u_E,u_W = reconstruction_sd2_2d(u)
    C0=c_flux_sd2_2d(u_N,u_S,u_E,u_W)
    u[j,j]+=C0
    boundary_conditions(u)
    u_N,u_S,u_E,u_W = reconstruction_sd2_2d(u)
    C1=c_flux_sd2_2d(u_N,u_S,u_E,u_W)
    u[j,j]+=0.5*(C1-C0)
    boundary_conditions(u)
    return u

In [ ]:
# SD3

j=slice(2,-2)
jp=slice(3,-1)
jm=slice(1,-3)

# p coefs: px, py, pdx, pdy

def px_coefs(u):
    pl0 = u[j,j].copy()
    pl1 = u[j,j]-u[jm,j]
    pr0 = pl0.copy()
    pr1 = u[jp,j]-u[j,j]

    pcx0 = pl0-1./12.*(u[jp,j]+u[j,jp]-4.*u[j,j]+u[jm,j]+u[j,jm])
    pcx1 = 0.5*(pl1+pr1)
    pcx2 = pr1-pl1
    return pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2

def py_coefs(u):
    pb0=u[j,j].copy()
    pb1=u[j,j]-u[j,jm]
    pt0=pb0.copy()
    pt1=u[j,jp]-u[j,j]

    pcy0=pb0-1./12.*(u[jp,j]+u[j,jp]-4.*u[j,j]+u[jm,j]+u[j,jm])
    pcy1=0.5*(pb1+pt1)
    pcy2=pt1-pb1
    return pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2

def pdx_coefs(u):
    pl0 = u[j,j].copy()
    pl1 = u[j,j]-u[jm,jm]
    pr0 = pl0.copy()
    pr1 = u[jp,jp]-u[j,j]

    pcx0 = pl0-1./12.*(u[jp,jp]+u[jm,jp]-4.*u[j,j]+u[jm,jm]+u[jp,jm])
    pcx1 = 0.5*(pl1+pr1)
    pcx2 = pr1-pl1
    return pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2

def pdy_coefs(u):
    pb0=u[j,j].copy()
    pb1=u[j,j]-u[jp,jm]
    pt0=pb0.copy()
    pt1=u[jm,jp]-u[j,j]

    pcy0=pb0-1./12.*(u[jp,jp]+u[jm,jp]-4.*u[j,j]+u[jm,jm]+u[jp,jm])
    pcy1=0.5*(pb1+pt1)
    pcy2=pt1-pb1
    return pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2

# indicators: indicators_2d_sd3, indicators_diag_2d_sd3

def indicators_2d_sd3(u):
    u_norm=np.sqrt(dx*dy)*np.linalg.norm(u[j,j])

    pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2 = px_coefs(u)
    
    ISl = pl1**2/(u_norm+eps)
    IScx = 1./(u_norm+eps)*((13./3.)*pcx2**2 + pcx1**2)
    ISr = pr1**2/(u_norm+eps)

    pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2 = py_coefs(u)

    ISb = pb1**2/(u_norm+eps)
    IScy = 1./(u_norm+eps)*((13./3.)*pcy2**2 + pcy1**2)
    ISt = pt1**2/(u_norm+eps)
    return ISl, IScx, ISr, ISb, IScy, ISt
    
def indicators_diag_2d_sd3(u):
    u_norm=np.sqrt(dx*dy)*np.linalg.norm(u[j,j])

    pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2 = pdx_coefs(u)

    dISl = pl1**2/(u_norm+eps)
    dIScx = 1./(u_norm+eps)*((13./3.)*pcx2**2 + pcx1**2)
    dISr = pr1**2/(u_norm+eps)    

    pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2 = pdy_coefs(u)

    dISb = pb1**2/(u_norm+eps)
    dIScy = 1./(u_norm+eps)*((13./3.)*pcy2**2 + pcy1**2)
    dISt = pt1**2/(u_norm+eps)
    return dISl, dIScx, dISr, dISb, dIScy, dISt

# reconstruction: reconstruction_2d_sd3, reconstruction_diag_2d_sd3

def reconstruction_2d_sd3(u,ISl, IScx, ISr, ISb, IScy, ISt):
    
    u_N,u_S,u_E,u_W = np.ones((4,)+u.shape)

    cl=.25; ccx=.5; cr=.25;    
    cb=.25; ccy=.5; ct=.25;    

    pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2 = px_coefs(u)

    alpl=cl/((eps+ISl)**2)
    alpcx=ccx/((eps+IScx)**2)
    alpr=cr/((eps+ISr)**2)
    alp_sum=alpl+alpcx+alpr
    wl=alpl/alp_sum
    wcx=alpcx/alp_sum
    wr=alpr/alp_sum

    pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2 = py_coefs(u)

    alpb=cb/((eps+ISb)**2)
    alpcy=ccy/((eps+IScy)**2)
    alpt=ct/((eps+ISt)**2)
    alp_sum=alpb+alpcy+alpt
    wb=alpb/alp_sum
    wcy=alpcy/alp_sum
    wt=alpt/alp_sum

    u_N[j,j] = wb*(pb0+.5*pb1)+wcy*(pcy0+.5*pcy1+.25*pcy2)+wt*(pt0+.5*pt1)
    u_S[j,j] = wb*(pb0-.5*pb1)+wcy*(pcy0-.5*pcy1+.25*pcy2)+wt*(pt0-.5*pt1)    
    u_E[j,j] = wl*(pl0+.5*pl1)+wcx*(pcx0+.5*pcx1+.25*pcx2)+wr*(pr0+.5*pr1)
    u_W[j,j] = wl*(pl0-.5*pl1)+wcx*(pcx0-.5*pcx1+.25*pcx2)+wr*(pr0-.5*pr1)    

    return u_N,u_S,u_E,u_W

def reconstruction_diag_2d_sd3(u, dISl, dIScx, dISr, dISb, dIScy, dISt):
    u_NE,u_SE,u_NW,u_SW = np.ones((4,)+u.shape)

    cl=.25; ccx=.5; cr=.25;    
    cb=.25; ccy=.5; ct=.25;    

    pl0, pl1, pr0, pr1, pcx0, pcx1, pcx2 = pdx_coefs(u)

    alpl=cl/(eps+dISl)**2
    alpcx=ccx/(eps+dIScx)**2
    alpr=cr/(eps+dISr)**2
    alp_sum=alpl+alpcx+alpr
    wl=alpl/alp_sum
    wcx=alpcx/alp_sum
    wr=alpr/alp_sum

    pb0, pb1, pt0, pt1, pcy0, pcy1, pcy2 = pdy_coefs(u)

    alpb=cb/(eps+dISb)**2
    alpcy=ccy/(eps+dIScy)**2
    alpt=ct/(eps+dISt)**2
    alp_sum=alpb+alpcy+alpt
    wb=alpb/alp_sum
    wcy=alpcy/alp_sum
    wt=alpt/alp_sum

    u_NW[j,j] = wb*(pb0+.5*pb1) + wcy*(pcy0+.5*pcy1+.25*pcy2) +wt*(pt0+.5*pt1)
    u_SE[j,j] = wb*(pb0-.5*pb1) + wcy*(pcy0-.5*pcy1+.25*pcy2) +wt*(pt0-.5*pt1)    
    u_NE[j,j] = wl*(pl0+.5*pl1) + wcx*(pcx0+.5*pcx1+.25*pcx2) +wr*(pr0+.5*pr1)
    u_SW[j,j] = wl*(pl0-.5*pl1) + wcx*(pcx0-.5*pcx1+.25*pcx2) +wr*(pr0-.5*pr1)    

    return u_NW,u_SE,u_NE,u_SW

# numerical fluxes: Hx_flux_2d_sd3, Hy_flux_2d_sd3, c_flux_2d_sd3

def Hx_flux_2d_sd3(u_NW, u_W, u_SW, u_NE, u_E, u_SE):
    a = np.maximum(spectral_radius_x(u_E),spectral_radius_x(u_W))
    f_E = flux_x(u_E)
    f_W = flux_x(u_W)
    f_NE = flux_x(u_NE)
    f_NW = flux_x(u_NW)
    f_SE = flux_x(u_SE)
    f_SW = flux_x(u_SW)

    Hx=1./12.*((f_NW+f_NE+4.*(f_W+f_E)+f_SW+f_SE) - a*(u_NW-u_NE+4.*(u_W-u_E)+u_SW-u_SE))
    return Hx
    
def Hy_flux_2d_sd3(u_SW, u_S, u_SE, u_NE, u_N, u_NW):
    b = np.maximum(spectral_radius_y(u_N),spectral_radius_y(u_S))
    g_N = flux_y(u_N)
    g_S = flux_y(u_S)
    g_NE = flux_y(u_NE)
    g_NW = flux_y(u_NW)
    g_SE = flux_y(u_SE)
    g_SW = flux_y(u_SW)

    Hy=1./12.*((g_SW+g_NW+4.*(g_S+g_N)+g_SE+g_NE) - b*(u_SW-u_NW+4.*(u_S-u_N)+u_SE-u_NE))
    return Hy

def c_flux_2d_sd3(u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW):
    Hx_halfm = Hx_flux_2d_sd3(u_NW[j,j], u_W[j,j], u_SW[j,j], u_NE[jm,j], u_E[jm,j], u_SE[jm,j])
    Hx_halfp = Hx_flux_2d_sd3(u_NW[jp,j], u_W[jp,j], u_SW[jp,j], u_NE[j,j], u_E[j,j], u_SE[j,j])
    Hy_halfm = Hy_flux_2d_sd3(u_SW[j,j], u_S[j,j], u_SE[j,j], u_NE[j,jm], u_N[j,jm], u_NW[j,jm])
    Hy_halfp = Hy_flux_2d_sd3(u_SW[j,jp], u_S[j,jp], u_SE[j,jp], u_NE[j,j], u_N[j,j], u_NW[j,j])
    return -dt/dx*(Hx_halfp-Hx_halfm) - dt/dy*(Hy_halfp-Hy_halfm)

# final scheme sd3_2d

def sd3_2d(u):
    # The commented out boundary conditions might actually be a bug in CentPack
    boundary_conditions(u)
    ISl, IScx, ISr, ISb, IScy, ISt = indicators_2d_sd3(u)
    u_N,u_S,u_E,u_W = reconstruction_2d_sd3(u, ISl, IScx, ISr, ISb, IScy, ISt)
    dISl, dIScx, dISr, dISb, dIScy, dISt = indicators_diag_2d_sd3(u)
    u_NW,u_SE,u_NE,u_SW=reconstruction_diag_2d_sd3(u, dISl, dIScx, dISr, dISb, dIScy, dISt)
    list(map(boundary_conditions,[u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW]))
    C0=c_flux_2d_sd3(u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW)
    u[j,j]+=C0
    boundary_conditions(u)
    u_N,u_S,u_E,u_W = reconstruction_2d_sd3(u, ISl, IScx, ISr, ISb, IScy, ISt)
    u_NW,u_SE,u_NE,u_SW=reconstruction_diag_2d_sd3(u, dISl, dIScx, dISr, dISb, dIScy, dISt)
    list(map(boundary_conditions,[u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW]))
    C1=c_flux_2d_sd3(u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW)
    u[j,j]+=0.25*(C1-3.*C0)
    boundary_conditions(u)
    u_N,u_S,u_E,u_W = reconstruction_2d_sd3(u, ISl, IScx, ISr, ISb, IScy, ISt)
    u_NW,u_SE,u_NE,u_SW=reconstruction_diag_2d_sd3(u, dISl, dIScx, dISr, dISb, dIScy, dISt)
    list(map(boundary_conditions,[u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW]))
    C2=c_flux_2d_sd3(u_N,u_S,u_E,u_W,u_NE,u_SE,u_SW,u_NW)
    u[j,j] += + 1./12.*(8.*C2-C1-C0)
    boundary_conditions(u)
    return u


### Examples

#### Scalar (SD3)

We solve the nonlinear scalar conservation law 

\begin{equation}
\partial_t u + \partial_x \sin u + \frac{1}{3} \partial_y u^3= 0,
\end{equation}

on the domain $(x,y,t)\in([0,2\pi]\times[0,2\pi]\times[0,6])$ with initial data

\begin{equation}
u(x,y,0) = \sin \left(x+\frac{1}{2}\right) \cos(2x+y)
\end{equation}

and periodic boundary conditions. The solution is computed using a 144 $\times$ 144 mesh and CFL number 0.9.

In [ ]:
# Parameters

x_init, x_final = 0.0, 6.283  
y_init, y_final = 0.0, 6.283
J = 144
K = 144
L=1
gamma=1.666666667
t_final = 6.0
dt_out = 0.1
cfl = 0.9
alpha = 1.4 

# Space grid
dx=(x_final - x_init)/J
dy=(y_final - y_init)/K
x = np.linspace(x_init-2*dx, y_final+dx, J+4)
y = np.linspace(y_init-2*dx, y_final+dx, K+4)
xx, yy = np.meshgrid(x, y, sparse=True)

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

eps=0.000001 # for SD3

In [ ]:
# Example function definitons

def initial_scalar_2d(): 
    return np.sin(xx.T+0.5)*np.cos(2*xx.T+yy.T)

def flux_x_scalar_2d(u):
    return np.sin(u)

def flux_y_scalar_2d(u):
    return 1./3.*u**3
    
def bc_periodic_2d(u):
    j=slice(2,-2)
    # x-boundary
    u[0] = u[-4]
    u[1] = u[-3]
    u[-2] = u[2]
    u[-1] = u[3]
    # y-boundary
    u[:,0] = u[:,-4]
    u[:,1] = u[:,-3]
    u[:,-2] = u[:,2]
    u[:,-1] = u[:,3]    

def bc_periodic_staggered_2d(u):
    if(odd):
        j=slice(1,-2)
        # y-boundary
        u[j,0] = u[j,-4]
        u[j,-2] = u[j,2]
        u[j,-1] = u[j,3]
        # x-boundary
        u[0] = u[-4]
        u[-2] = u[2]
        u[-1] = u[3]
    else:
        j=slice(2,-1)        
        # y-boundary
        u[j,0] = u[j,-4]
        u[j,1] = u[j,-3]
        u[j,-1] = u[j,3]
        # x-boundary
        u[0] = u[-4]
        u[1] = u[-3]
        u[-1] = u[3]    
    
def spectral_radius_x_scalar_2d(u):
    return np.abs(np.cos(u))

def spectral_radius_y_scalar_2d(u):
    return u**2

def spectral_radii(u):
    return np.abs(np.cos(u)), u**2

# Example function assignments

initial=initial_scalar_2d
flux_x=flux_x_scalar_2d
flux_y=flux_y_scalar_2d
boundary_conditions=bc_periodic_2d
spectral_radius_x=spectral_radius_x_scalar_2d
spectral_radius_y=spectral_radius_y_scalar_2d

In [ ]:
# Simulation
solver=sd3_2d

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max_x=np.max(spectral_radius_x(u))
    r_max_y=np.max(spectral_radius_y(u))
    dt = cfl/np.sqrt((r_max_x/dx)**2+(r_max_y/dy)**2)
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation

fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))

ax.set_title("Nonlinear scalar")
ax.set_xlabel("x")
ax.set_ylabel("y")

contours=ax.contour(x[j], y[j], u_n[0,j,j], 8, colors='black') 
img=ax.imshow(u_n[0,j,j], extent=[0, 6.3, 0, 6.3], origin='lower',
           cmap='ocean', alpha=0.5)
#cl=ax.clabel(contours, inline=True, fontsize=8)

fig.colorbar(img)
plt.axis(aspect='image')
def animate(i):
    ax.collections = []
    ax.contour(x[j], y[j], u_n[i,j,j], 8, colors='black') 
#    cl=ax.clabel(contours, inline=True, fontsize=8)
    img.set_array(u_n[i,j,j])
    img.autoscale()
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=100, blit=False)
 
plt.show()

#### Euler (FD2)

We solve the Euler equations in 2D 

\begin{equation} 
\partial_t 
\begin{bmatrix} \rho \\ \rho u_x \\ rho u_y \\ E \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} \rho u_x \\ \rho u_x^2 + p \\  \rho u_x u_y \\ (E+p) u_x \end{bmatrix} 
+ 
\partial_y 
\begin{bmatrix} \rho u_y \\ \rho u_y u_x \\  \rho u_y^2 +p \\ (E+p) u_y \end{bmatrix} 
= 0 
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (u_x^2 - u_y^2) \right), \qquad \gamma=1.4
\end{equation}

on the domain $(x,y,t)\in([0,1]\times[0,1]\times[0,0.1])$ with initial data for a *2D Riemann problem*:

\begin{equation}
(\rho, v, p)_{t=0} = 
\begin{cases}
(1,0,1) & \text{if} & 0<x\leq0.5 \\
(0.125, 0, 0.1) & \text{if} & 0.5<x<1
\end{cases}
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using a 200 $\times$ 200 mesh and CFL number 0.75.

In [ ]:
# Parameters

x_init, x_final = 0.0, 1.0 
y_init, y_final = 0.0, 1.0 
J = 200 
K = 200 
L=4
gamma=1.4
t_final = 0.4
dt_out = 0.005
cfl = 0.475
alpha = 1.4 

# Space grid
dx=(x_final - x_init)/J
dy=(y_final - y_init)/K
x = np.linspace(x_init-2*dx, y_final+dx, J+4)
y = np.linspace(y_init-2*dx, y_final+dx, K+4)

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

# Scheme-specific parameters
odd=True # for FD2
eps=0.000001 # for SD3

In [ ]:
# Example functions

def pressure(u):
    return (gamma-1.0)*(u[:,:,3]-0.5*(u[:,:,1]**2+u[:,:,2]**2)/u[:,:,0])

def euler_data():

    p_one=1.5
    p_two=0.3
    p_three=0.029
    p_four=0.3
      
    upper_right,upper_left,lower_right,lower_left = np.ones((4,4))
    
    upper_right[0] = 1.5
    upper_right[1] = 0.0
    upper_right[2] = 0.0
    upper_right[3] = p_one/(gamma - 1.0) + 0.5*(upper_right[1]**2 + upper_right[2]**2)/upper_right[0]

    upper_left[0] = 0.5323
    upper_left[1] = 1.206*upper_left[0]
    upper_left[2] = 0.0
    upper_left[3] = p_two/(gamma - 1.0) + 0.5*(upper_left[1]**2 + upper_left[2]**2)/upper_left[0]

    lower_right[0] = 0.5323
    lower_right[1] = 0.0
    lower_right[2] = 1.206*lower_right[0]
    lower_right[3] = p_four/(gamma - 1.0) + 0.5*(lower_right[1]**2 + lower_right[2]**2)/lower_right[0]

    lower_left[0] = 0.138
    lower_left[1] = 1.206*lower_left[0]
    lower_left[2] = 1.206*lower_left[0]
    lower_left[3] = p_three/(gamma - 1.0) + 0.5*(lower_left[1]**2 + lower_left[2]**2)/lower_left[0]

    return upper_right,upper_left,lower_right,lower_left

def initial_euler_2d(): 
    u=np.empty((J+4,K+4,L))
    midJ = int(J/2)+2
    midK = int(K/2)+2
    if (L!=4): print("Euler in 2D needs 4 variables (L=4).")

    one_matrix = np.ones(u[midJ:,midK:].shape)
    upper_right,upper_left,lower_right,lower_left = euler_data()
    
    u[midJ:,midK:] = upper_right*one_matrix
    u[:midJ,midK:] = upper_left*one_matrix
    u[midJ:,:midK] = lower_right*one_matrix
    u[:midJ,:midK] = lower_left*one_matrix
    return u

def flux_x_euler_2d(u):
    f = np.empty_like(u)
    
    p=pressure(u)

    f[:,:,0]=u[:,:,1]
    f[:,:,1]=u[:,:,1]**2/u[:,:,0] + p
    f[:,:,2]=u[:,:,1]*u[:,:,2]/u[:,:,0]
    f[:,:,3]=(u[:,:,3]+p)*u[:,:,1]/u[:,:,0]  
    
    return f

def flux_y_euler_2d(u):
    g = np.empty_like(u)

    p=pressure(u)

    g[:,:,0]=u[:,:,2]
    g[:,:,1]=u[:,:,1]*u[:,:,2]/u[:,:,0]
    g[:,:,2]=u[:,:,2]**2/u[:,:,0] + p
    g[:,:,3]=(u[:,:,3]+p)*u[:,:,2]/u[:,:,0]  
    
    return g                   
                   
def bc_euler_2d(u):

    upper_right,upper_left,lower_right,lower_left = euler_data()
    
    j=slice(2,-2)    

    u[j,0] = u[j,2]
    u[j,1] = u[j,2]
    u[j,-2]= u[j,-3]
    u[j,-1]= u[j,-3]
    
    u[0,j] = u[2,j]
    u[1,j] = u[2,j]
    u[-2,j]= u[-3,j]
    u[-1,j]= u[-3,j]
    
    # one
    u[-2,-2] = upper_right 
    u[-1,-2] = upper_right
    u[-2,-1] = upper_right
    u[-1,-1] = upper_right

    # two
    u[0,-2] = upper_left
    u[0,-1] = upper_left 
    u[1,-2] = upper_left 
    u[1,-1] = upper_left 
    
    # three
    u[0,0] = lower_left
    u[0,1] = lower_left
    u[1,0] = lower_left
    u[1,1] = lower_left

    # four
    u[-2,0] = lower_right
    u[-1,0] = lower_right
    u[-2,1] = lower_right
    u[-1,1] = lower_right

def bc_euler_staggered_2d(u):

    upper_right,upper_left,lower_right,lower_left = euler_data()
    
    if(odd):
        j=slice(1,-2)
        u[j,0] = u[j,1]
        u[j,-2]= u[j,-3]
        u[j,-1]= u[j,-3]

        u[0,j] = u[1,j]
        u[-2,j]= u[-3,j]
        u[-1,j]= u[-3,j]

         # one
        u[-2,-2] = upper_right 
        u[-1,-2] = upper_right
        u[-2,-1] = upper_right
        u[-1,-1] = upper_right
 
        # two
        u[0,-2] = upper_left
        u[0,-1] = upper_left 

        # three
        u[0,0] = lower_left
        u[0,1] = lower_left
        u[1,0] = lower_left
        u[1,1] = lower_left    

        # four
        u[-2,0] = lower_right
        u[-1,0] = lower_right
        u[-2,1] = lower_right
        u[-1,1] = lower_right

    else:
        
        j=slice(2,-1)
        u[j,0] = u[j,2]
        u[j,1] = u[j,2]
        u[j,-1]= u[j,-2]

        u[0,j] = u[2,j]
        u[1,j] = u[2,j]
        u[-1,j]= u[-2,j]

        # one
        u[-1,-2] = upper_right 
        u[-1,-1] = upper_right
 
        # two
        u[0,-2] = upper_left
        u[0,-1] = upper_left 
        u[1,-2] = upper_left 
        u[1,-1] = upper_left 

        # three
        u[0,0] = lower_left
        u[0,1] = lower_left
        u[1,0] = lower_left
        u[1,1] = lower_left

        # four
        u[-1,0] = lower_right 
        u[-1,1] = lower_right                
        
def spectral_radius_x_euler_2d(u):
    rho=u[j,j,0]
    vx=u[j,j,1]/rho
    vy=u[j,j,2]/rho
    p=(gamma-1.0)*(u[j,j,3]-.5*rho*(vx**2 + vy**2))
    c=np.sqrt(gamma*p/rho)
    return np.abs(vx)+c

def spectral_radius_y_euler_2d(u):
    rho=u[j,j,0]
    vx=u[j,j,1]/rho
    vy=u[j,j,2]/rho
    p=(gamma-1.0)*(u[j,j,3]-.5*rho*(vx**2 + vy**2))
    c=np.sqrt(gamma*p/rho)
    return np.abs(vy)+c

# Example function assignments

initial=initial_euler_2d
flux_x=flux_x_euler_2d
flux_y=flux_y_euler_2d
boundary_conditions=bc_euler_staggered_2d
spectral_radius_x=spectral_radius_x_euler_2d
spectral_radius_y=spectral_radius_y_euler_2d

In [ ]:
# Simulation

solver=fd2_2d

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max_x=np.max(spectral_radius_x(u))
    r_max_y=np.max(spectral_radius_y(u))
    dt = cfl/np.sqrt((r_max_x/dx)**2+(r_max_y/dy)**2)
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation

fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))
#levels=np.arange(-1, 1, 0.1) 

ax.contour(x[1:-1], y[1:-1], u_n[0,1:-1,1:-1,0]) 
def animate(i):
    ax.collections = []
    ax.contour(x[1:-1], y[1:-1], u_n[i,1:-1,1:-1,0])
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=100, blit=False)
 
plt.show()

#### MHD (SD2)

We solve the equations of ideal magnetohydrodynamics in 1D 

\begin{equation} 
\partial_t 
\begin{bmatrix} 
\rho \\ \rho v_x \\ \rho v_y \\ \rho v_z \\ B_x \\ B_y \\ B_z \\ E 
\end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} 
\rho v_x \\ \rho v_x^2 + p^* - B_x^2 \\ \rho v_x v_y - B_x B_y \\
\rho v_x v_z - B_x B_z \\ 0 \\ B_y v_x - B_x v_y \\ B_z v_x - B_x v_z \\
(E+p^*) v_x - B_x (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
+ 
\partial_y
\begin{bmatrix} 
\rho v_y \\ \rho v_y v_x - B_y B_x \\ \rho v_y^2 + p^* - B_y^2  \\
\rho v_y v_z - B_y B_z \\ B_x v_y - B_y v_x \\ 0 \\ B_z v_y - B_y v_z \\
(E+p^*) v_y - B_y (B_x v_x + B_y v_y + B_z v_Z) 
\end{bmatrix} 
= 0 
\end{equation}

where the total pressure is given by 

\begin{equation}
p^* = p + \frac{1}{2} (B_x^2 + B_y^2 + B_z^2)
\end{equation}

with the equation of state

\begin{equation}
p = (\gamma-1) \left(E-\frac{1}{2} \rho (v_x^2+v_y^2+v_z^2) - 
\frac{1}{2}(B_x^2 + B_y^2 + B_z^2)\right), \qquad \gamma=5/3
\end{equation}

The solution is computed on the domain $(x,y,t)\in([0,2\pi]\times[0,2\pi]\times[0,3])$ with initial data for an *Orszag-Tang vortex system*:

\begin{equation}
(\rho, v_x, v_y, v_z, B_x, B_y, B_z, p)_{t=0} = 
(\gamma^2, -\sin y, \sin x, 0, -\sin y, \sin (2x), 0, \gamma)
\end{equation}

and Dirichlet boundary data set by initial data on each boundary. The solution is computed using a 288$\times$288 mesh and CFL number 0.75.

In [ ]:
# Parameters

x_init, x_final = 0.0, 6.283 #2.*np.pi 
y_init, y_final = 0.0, 6.283 #2.*np.pi
J = 144
K = 144
L=8
gamma=1.666666667
t_final = 3.0
dt_out = 0.05
cfl = 0.75
alpha = 1.3 

# Space grid
dx=(x_final - x_init)/J
dy=(y_final - y_init)/K
x = np.linspace(x_init-2*dx, y_final+dx, J+4)
y = np.linspace(y_init-2*dx, y_final+dx, K+4)
xx, yy = np.meshgrid(x, y, sparse=True)

# Time grid for stored time steps
Nt = int(np.ceil(t_final/dt_out))

In [ ]:
# Example functions

def pressure(u):
    return (gamma-1.0)*(u[:,:,7]-0.5*(u[:,:,1]**2+u[:,:,2]**2+u[:,:,3]**2)/u[:,:,0]\
                     -0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))

def pressure_star(u):
    return pressure(u) + 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2) 


def initial_mhd_2d(): 
    u=np.empty((J+4,K+4,L))
    # The transform below is due to the way numpy stores meshgrids
    y=yy.T
    x=xx.T
    if (L!=8): print("MHD in 2D needs 8 variables (L=8).")
    
    u[:,:,0] = gamma**2
    u[:,:,1] = u[:,:,0]/dy*(np.cos(y+0.5*dy)-np.cos(y-0.5*dy))
    u[:,:,2] = 0.
    u[:,:,3] = -u[:,:,0]/dx*(np.cos(x+0.5*dx)-np.cos(x-0.5*dx))
    u[:,:,4] = 1./dy*(np.cos(y+0.5*dy)-np.cos(y-0.5*dy))
    u[:,:,5] = 0.
    u[:,:,6] = -0.5/dx*(np.cos(2.*(x+0.5*dx))-np.cos(2.*(x-0.5*dx)))
    
    I1 =-0.125/dy*(u[:,:,0]+1.)*(np.sin(2.*(y+0.5*dy))-np.sin(2.*(y-0.5*dy)))
    I2 =-0.125/dx*u[:,:,0]*(np.sin(2.*(x+0.5*dx))-np.sin(2.*(x-0.5*dx)))
    I3 =-0.0625/dx*(np.sin(4.*(x+0.5*dx))-np.sin(4.*(x-0.5*dx)))
    u[:,:,7] = 3.+0.5*u[:,:,0]+I1+I2+I3    

    return u

def flux_x_mhd_2d(u):
    f = np.empty_like(u)
    
    p_star=pressure_star(u)    
    
    f[:,:,0]=u[:,:,1]
    f[:,:,1]=u[:,:,1]**2/u[:,:,0] + p_star - u[:,:,4]**2
    f[:,:,2]=u[:,:,1]*u[:,:,2]/u[:,:,0] - u[:,:,4]*u[:,:,5]
    f[:,:,3]=u[:,:,1]*u[:,:,3]/u[:,:,0] - u[:,:,4]*u[:,:,6]
    f[:,:,4]=0.
    f[:,:,5]=u[:,:,1]*u[:,:,5]/u[:,:,0] - u[:,:,4]*u[:,:,2]/u[:,:,0]
    f[:,:,6]=u[:,:,1]*u[:,:,6]/u[:,:,0] - u[:,:,4]*u[:,:,3]/u[:,:,0]
    f[:,:,7]=(u[:,:,7]+p_star)*u[:,:,1]/u[:,:,0] - u[:,:,4]*(u[:,:,4]*u[:,:,1]/u[:,:,0]+u[:,:,5]*u[:,:,2]/u[:,:,0]+\
                                                             u[:,:,6]*u[:,:,3]/u[:,:,0])

    return f

def flux_y_mhd_2d(u):
    g = np.empty_like(u)

    p_star=pressure_star(u)

    g[:,:,0]=u[:,:,3]
    g[:,:,1]=u[:,:,3]*u[:,:,1]/u[:,:,0] - u[:,:,4]*u[:,:,6]
    g[:,:,2]=u[:,:,3]*u[:,:,2]/u[:,:,0] - u[:,:,5]*u[:,:,6]
    g[:,:,3]=u[:,:,3]**2/u[:,:,0] + p_star - u[:,:,6]**2

    g[:,:,4]=u[:,:,3]*u[:,:,4]/u[:,:,0] - u[:,:,6]*u[:,:,1]/u[:,:,0]
    g[:,:,5]=u[:,:,3]*u[:,:,5]/u[:,:,0] - u[:,:,6]*u[:,:,2]/u[:,:,0]
    g[:,:,6]=0.
    g[:,:,7]=(u[:,:,7]+p_star)*u[:,:,3]/u[:,:,0] - u[:,:,6]*(u[:,:,4]*u[:,:,1]/u[:,:,0]+u[:,:,5]*u[:,:,2]/u[:,:,0]+\
                                                             u[:,:,6]*u[:,:,3]/u[:,:,0])    
   
    return g                   
                   
def bc_mhd_2d(u): # periodic (come back if there's a problem, you did not restrict to j 2,-2 for the y-boundary)
    # x-boundary
    u[0] = u[-4]
    u[1] = u[-3]
    u[-2] = u[2]
    u[-1] = u[3]
    # y-boundary
    u[:,0] = u[:,-4]
    u[:,1] = u[:,-3]
    u[:,-2] = u[:,2]
    u[:,-1] = u[:,3]    
        
def spectral_radius_x_mhd_2d(u):   
    rho=u[:,:,0]
    vx=u[:,:,1]/rho
    vy=u[:,:,2]/rho
    vz=u[:,:,3]/rho
    p=(gamma-1.0)*(u[:,:,7]-.5*rho*(vx**2 + vy**2 + vz**2) - 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))
    A=gamma*p/rho
    B=(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2)/rho
    cfx=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,4]**2/rho)))
    cfy=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,6]**2/rho)))

    return np.abs(vx)+cfx

def spectral_radius_y_mhd_2d(u):
    rho=u[:,:,0]
    vx=u[:,:,1]/rho
    vy=u[:,:,2]/rho
    vz=u[:,:,3]/rho
    p=(gamma-1.0)*(u[:,:,7]-.5*rho*(vx**2 + vy**2 + vz**2) - 0.5*(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2))
    A=gamma*p/rho
    B=(u[:,:,4]**2+u[:,:,5]**2+u[:,:,6]**2)/rho
    cfx=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,4]**2/rho)))
    cfy=np.sqrt(0.5*(A+B+np.sqrt((A+B)**2-4*A*u[:,:,6]**2/rho)))

    return np.abs(vy)+cfy

# Example function assignments

initial=initial_mhd_2d
flux_x=flux_x_mhd_2d
flux_y=flux_y_mhd_2d
boundary_conditions=bc_mhd_2d
spectral_radius_x=spectral_radius_x_mhd_2d
spectral_radius_y=spectral_radius_y_mhd_2d

In [ ]:
# Simulation

solver=sd2_2d

u = initial()
u_n = np.empty((Nt+1,)+u.shape)  # output array
u_n[0] = u.copy()

i=0; t=0.; t_out=0.
while t<t_final:
    r_max_x=np.max(spectral_radius_x(u))
    r_max_y=np.max(spectral_radius_y(u))
    dt = cfl/np.sqrt((r_max_x/dx)**2+(r_max_y/dy)**2)
    dt=min(dt,dt_out-t_out)
    t+=dt; t_out+=dt;
    solver(u)
    # Store if t_out=dt_out
    if (t_out == dt_out):
        i+=1
        u_n[i,:] = u
        t_out=0

In [ ]:
# Animation

fig = plt.figure()
ax = plt.axes(xlim=(x_init,x_final), ylim=(y_init, y_final))

ax.set_title("MHD Density")
ax.set_xlabel("x")
ax.set_ylabel("y")

#contours=ax.contour(x[j], y[j], u_n[0,j,j,0], 6, colors='black') 
img=ax.imshow(u_n[0,j,j,0], extent=[0, 6.3, 0, 6.3], origin='lower',
           cmap='inferno', alpha=0.5)
#cl=ax.clabel(contours, inline=True, fontsize=8)

fig.colorbar(img)
plt.axis(aspect='image')
def animate(i):
    ax.collections = []
    ax.contour(x[j], y[j], u_n[i,j,j,0], 20, colors='black',linewidths=0.1) 
    img.set_array(u_n[i,j,j,0])
    img.autoscale()
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=50, blit=False)
 
plt.show()

In [ ]:
# Wireframe

from mpl_toolkits.mplot3d import axes3d

fig = plt.figure()
ax = axes3d.Axes3D(fig)

wframe = ax.plot_wireframe(xx, yy, u_n[0,:,:,0], rstride=2, cstride=2)
#ax.set_zlim(-1,1)

def update(i, ax, fig):
    ax.cla()
    wframe = ax.plot_wireframe(xx,yy,u_n[i,:,:,0], rstride=2, cstride=2)
    #ax.set_zlim(-1,1)
    return wframe,

ani = animation.FuncAnimation(fig, update, 
        frames=Nt, 
        fargs=(ax, fig), interval=100)
plt.show()